# Librerías

In [1]:
import time
import re
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import warnings 
warnings.filterwarnings('ignore')

C:\Users\epile\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Funciones

In [2]:
def sumar_numeros_rango(df, columna_texto):
    def extraer_y_sumar(text):
        numeros = re.findall(r'\d+(?:\.\d{1,2})?', text)
        numeros_en_rango = [float(num) for num in numeros if 50 <= float(num) <= 50000]
        return sum(numeros_en_rango) if numeros_en_rango else np.nan
    df['INGRESO'] = df[columna_texto].apply(lambda x: extraer_y_sumar(str(x)))
    
def get_embedding(texts):
    EMBEDDING_MODEL = "all-MiniLM-L6-v2"
    model = SentenceTransformer(EMBEDDING_MODEL)
    embedding = model.encode(texts, convert_to_numpy=True) 
    return embedding 

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_a = np.linalg.norm(vec1)
    norm_b = np.linalg.norm(vec2)
    return dot_product / (norm_a * norm_b)

# Limpieza de Datos

In [25]:
data = pd.read_csv('data/REPARACIONES.csv', encoding='latin1')
clientes = pd.read_csv('data/CLIENTES.csv', encoding='latin1')

sumar_numeros_rango(data, 'INFORMETALLER')
data_cleaned = data.dropna(subset=['INGRESO'])

data_cleaned["COMBINADO"] = (
    data_cleaned["FALLA"].fillna(" ") + " " +
    data_cleaned["OBSERVACIONES"].fillna(" ") + " " +
    data_cleaned["INFORMETALLER"].fillna(" ") )

data_cleaned.reset_index(inplace=True)

data_cleaned["COMBINADO"] = (
    data_cleaned["COMBINADO"]
    .str.replace(r"[^a-zA-Z\s]", "", regex=True)
    .str.lower())
data_cleaned = data_cleaned[['CLIENTE', 'COMBINADO', 'INGRESO']]
data_cleaned.head(10)

,CLIENTE,COMBINADO,INGRESO
0,2,revision general tndr cambio del panel de cont...,1820.0
1,12,incluir el editor de pdf contasea hienz inc...,150.0
2,2,imprime negro tn mantenimiento iva,480.0
3,3,mayusculas no escribe minusculasse nota mas ...,360.0
4,3,rev gral margarita cinta tapa y cubierta rep...,360.0
5,2,rev gral cartucho toner reparacion traccion de...,1030.0
6,2,rev gral sin toner con cable de energia bue...,530.0
7,2,rev gral sin toner cambio de contacto del ca...,480.0
8,3,rev gral toner reparacion de traccion de papel...,480.0
9,4,rev gral toners el juego de cartuchos que envi...,400.0


# Creación de Embeddings

In [4]:
keywords = {
    'Impresora Laser': ['impresora laser', 'tn', 'toner', 'drum', 'tndr', 'dr', 'dl', 'fusor', 'pick up', 'pick', 'up',
                        'calor', 'presion', 'pad', 'termistor', 'toners', 'electroiman', 'lampara'],
    'Impresora Tinta': ['impresora tinta', 'reset', 'almohadillas', 'bateria', 'encoder', 'tinta',
                        '12v', 'cinta', 'perilla', 'cabezal', 'manguera', 'mascarilla'],
    'Sistemas': ['sistema', 'office', 'pantalla', 'windows', 'bisagra', 'teclado', 
                 'mouse', 'mousepad', 'ssd', 'operativo', 'disco duro', 'mayusculas', 'minusculas',
                 'ram', 'procesador', 'madre', 'correo', 'gabinete', 'bios', 'bocina'],
    'Otros': ['otros', 'margarita', 'myler', 'mylar', 'videocasetera', 'corrector']
}

In [5]:
inicio = time.time()
################################################################

imp_laser = []
imp_tinta = []
sistemas = []
otros = []

for palabra in keywords['Impresora Laser']:
    embedding = get_embedding(palabra)
    imp_laser.append(embedding)

for palabra in keywords['Impresora Tinta']:
    embedding = get_embedding(palabra)
    imp_tinta.append(embedding)

for palabra in keywords['Sistemas']:
    embedding = get_embedding(palabra)
    sistemas.append(embedding)

for palabra in keywords['Otros']:
    embedding = get_embedding(palabra)
    otros.append(embedding)


keywords_embedding = {
    'Impresora Laser':imp_laser,
    'Impresora Tinta':imp_tinta,
    'Sistemas':sistemas,
    'Otros':otros
} 

################################################################
fin = time.time()
tiempo_total = fin - inicio
print(f"El script tardó {tiempo_total:.4f} segundos en ejecutarse.")

El script tardó 81.5681 segundos en ejecutarse.


In [9]:
inicio = time.time()
################################################################

muestra_embedding = []
muestra = data_cleaned['COMBINADO'].head(100).reset_index()

for i in range(len(muestra)):
    embedding = get_embedding(muestra['COMBINADO'][i])
    muestra_embedding.append(embedding)


################################################################
fin = time.time()
tiempo_total = fin - inicio
print(f"El script tardó {tiempo_total:.4f} segundos en ejecutarse.")

El script tardó 153.5003 segundos en ejecutarse.


In [10]:
resultados_correlacion = []

for i, vector in enumerate(muestra_embedding):
    for key, lista_vectores in keywords_embedding.items():
        for j, vector_dict in enumerate(lista_vectores):
            cos_sim = cosine_similarity(vector, vector_dict)
            resultados_correlacion.append({
                "Número de falla": i + 1, 
                "Segmento": key,  
                "Palabra": keywords[key][j], 
                "Similitud Coseno": cos_sim
            })

df_resultados = pd.DataFrame(resultados_correlacion)

In [35]:
idx = df_resultados.groupby('Número de falla')['Similitud Coseno'].idxmax()
fallas = df_resultados.loc[idx].reset_index(drop=True)
fallas

,Número de falla,Segmento,Palabra,Similitud Coseno
0,1,Impresora Laser,tndr,0.298307
1,2,Sistemas,procesador,0.315053
2,3,Impresora Tinta,impresora tinta,0.391361
3,4,Sistemas,sistema,0.408414
4,5,Sistemas,teclado,0.479646
...,...,...,...,...
95,96,Impresora Tinta,cinta,0.613486
96,97,Impresora Tinta,bateria,0.649041
97,98,Sistemas,teclado,0.481906
98,99,Sistemas,sistema,0.305548


In [16]:
prueba = data_cleaned.head(100)
prueba['Cluster'] = fallas['Segmento']
prueba

,CLIENTE,COMBINADO,INGRESO,Cluster
0,2,revision general tndr cambio del panel de cont...,1820.0,Impresora Laser
1,12,incluir el editor de pdf contasea hienz inc...,150.0,Sistemas
2,2,imprime negro tn mantenimiento iva,480.0,Impresora Tinta
3,3,mayusculas no escribe minusculasse nota mas ...,360.0,Sistemas
4,3,rev gral margarita cinta tapa y cubierta rep...,360.0,Sistemas
...,...,...,...,...
95,196,cinta y corrector iva,380.0,Impresora Tinta
96,67,rev gral cambio de bateria v ah servicio iva,420.0,Impresora Tinta
97,68,entregada,2018.0,Sistemas
98,68,rep de mecanismo cambio de tarjeta logica...,880.0,Sistemas


In [28]:
clientes = clientes[['CLIENTE', 'COHORT']].head(100)
clientes

,CLIENTE,COHORT
0,0,21/11/2014
1,2,22/11/2017
2,3,22/11/2017
3,4,07/06/2024
4,5,27/04/2018
...,...,...
95,96,21/12/2017
96,97,04/12/2018
97,98,30/09/2024
98,99,21/12/2017


In [34]:
df_merged = pd.merge(prueba, clientes, on='CLIENTE', how='inner')
df_merged['COHORT'] = pd.to_datetime(df_merged['COHORT']).dt.to_period('M')
df_merged

,CLIENTE,COMBINADO,INGRESO,Cluster,COHORT
0,2,revision general tndr cambio del panel de cont...,1820.0,Impresora Laser,2017-11
1,2,imprime negro tn mantenimiento iva,480.0,Impresora Tinta,2017-11
2,2,rev gral cartucho toner reparacion traccion de...,1030.0,Impresora Laser,2017-11
3,2,rev gral sin toner con cable de energia bue...,530.0,Impresora Laser,2017-11
4,2,rev gral sin toner cambio de contacto del ca...,480.0,Impresora Laser,2017-11
...,...,...,...,...,...
81,66,atora las cintas cintas para pruebas revision...,100.0,Impresora Tinta,2017-08
82,67,rev gral cambio de bateria v ah servicio iva,420.0,Impresora Tinta,2019-02
83,68,entregada,2018.0,Sistemas,2018-08
84,68,rep de mecanismo cambio de tarjeta logica...,880.0,Sistemas,2018-08
